In [ ]:
from model_utils import *
from torch import optim
from constants import *
from model import *
from Data_utils import *

device = 'cuda'
# Creating the model from YOLOv3 class 
model = YOLOv3(in_channels=3,num_classes=3).to(device) 
  
# Defining the optimizer 
optimizer = optim.Adam(model.parameters(), lr = leanring_rate) 
  
# Defining the loss function 
loss_fn = YOLOLoss() 
  
# Defining the scaler for mixed precision training 
scaler = torch.cuda.amp.GradScaler() 
  
# Defining the train dataset 
train_dataset = Dataset( 
    csv_file="../../og/train.csv", 
    image_dir="../../og/images/", 
    label_dir="../../og/labels/", 
    grid_sizes=[19,38,76],
    anchors=ANCHORS, 
    transform=train_transform 
) 
  
# Defining the train data loader 
train_loader = torch.utils.data.DataLoader( 
    train_dataset, 
    batch_size = 32, 
    num_workers = 1, 
    shuffle = True, 
    pin_memory = True, 
)

test_dataset = Dataset( 
    csv_file="../../og/train.csv", 
    image_dir="../../og/images/", 
    label_dir="../../og/labels/", 
    grid_sizes=[19,38,76],
    anchors=ANCHORS, 
    transform=test_transform 
)

# Defining the train data loader 
test_loader = torch.utils.data.DataLoader( 
    train_dataset, 
    batch_size = 1, 
    num_workers = 1, 
    shuffle = False, 
    pin_memory = True, 
)
  
# Scaling the anchors 
scaled_anchors = ( 
    torch.tensor(ANCHORS) * 
    torch.tensor(s).unsqueeze(1).unsqueeze(1).repeat(1,3,2) 
).to(device) 
  
# Training the model 
for e in range(1, 2): 
    print("Epoch:", e) 
    training_loop(train_loader, model, optimizer, loss_fn, scaler, scaled_anchors) 
    loss = evaluate(test_loader, model, loss_fn, scaled_anchors) 
    print("Loss:", loss)
    # Saving the model 
    if save_model: 
        save_checkpoint(model, optimizer, filename=f"checkpoint.pth.tar")

In [ ]:
# Taking a sample image and testing the model 
  
# Setting the load_model to True 
load_model = True
  
# Defining the model, optimizer, loss function and scaler 
model = YOLOv3(in_channels=3,num_classes=3).to(device) 
optimizer = optim.Adam(model.parameters(), lr = leanring_rate) 
loss_fn = YOLOLoss() 
scaler = torch.cuda.amp.GradScaler() 
  
# Loading the checkpoint 
if load_model: 
    load_checkpoint(checkpoint_file, model, optimizer, leanring_rate) 
  
# Defining the test dataset and data loader 
test_dataset = Dataset( 
    csv_file="../../og/train.csv", 
    image_dir="../../og/images/", 
    label_dir="../../og/labels/", 
    grid_sizes=[19,38,76],
    anchors=ANCHORS, 
    transform=test_transform 
) 
test_loader = torch.utils.data.DataLoader( 
    test_dataset, 
    batch_size = 1, 
    num_workers = 2, 
    shuffle = True, 
) 
  
# Getting a sample image from the test data loader 
x, y = next(iter(test_loader)) 
x = x.to(device) 
  
model.eval() 
with torch.no_grad(): 
    # Getting the model predictions 
    output = model(x) 
    # Getting the bounding boxes from the predictions 
    bboxes = [[] for _ in range(x.shape[0])] 
    anchors = ( 
            torch.tensor(ANCHORS) 
                * torch.tensor(s).unsqueeze(1).unsqueeze(1).repeat(1, 3, 2) 
            ).to(device) 
  
    # Getting bounding boxes for each scale 
    for i in range(3): 
        batch_size, A, S, _, _ = output[i].shape 
        anchor = anchors[i] 
        boxes_scale_i = convert_cells_to_bboxes( 
                            output[i], anchor, s=S, is_predictions=True
                        ) 
        for idx, (box) in enumerate(boxes_scale_i): 
            bboxes[idx] += box 
model.train() 

print(len(bboxes))
# Plotting the image with bounding boxes for each image in the batch 
for i in range(batch_size): 
    # Applying non-max suppression to remove overlapping bounding boxes 
    nms_boxes = nms(bboxes[i], iou_threshold=0.1, threshold=0.1) 
    # Plotting the image with bounding boxes 
    plot_image(x[i].permute(1,2,0).detach().cpu(), nms_boxes)